# Эксперименты по магистерской диссертации
Тема - обнаружение инсайдеров  
Датасеты - CERT 4.2, 6.2, в виде БД
### План работы:  
0) Привести набор данных к удобному виду  
1) Предобработка данных (генерация "предложений поведения пользователя")  
2) Предобработка контентных данных. Пока только письма  
3) Обучение трансформера типа Bert поведению пользователей  
4) Обучение классификатора пользователей  
5) Скор аномальности как ошибка классификатора  

In [ ]:
import json
import sqlite3
import csv
from datetime import datetime
import os
import csv
import logging
import numpy as np
import itertools
from tqdm import tqdm
from collections import namedtuple
import math
import pickle


class Device:
#     id,date,user,pc,activity
    col = {name:idx for idx, name in enumerate(["id","date","user","pc","activity"])}
    connect = 1
    disconnect = 0
    feature_len = 2
    
class Email:
#     id,date,user,pc,to,cc,bcc,from,size,attachments,content
    col = {name:idx for idx, name in enumerate(["id","date","user","pc","to","cc","bcc","from","size","attachments","content"])}
    feature_len = 3
    
class File:
#     id,date,user,pc,filename,content
    col = {name:idx for idx, name in enumerate(["id","date","user","pc","filename","content"])}
    feature_len = 1
    
    
class Http:
#     id,date,user,pc,url,content
    col = {name:idx for idx, name in enumerate(["id","date","user","pc","url","content"])}
    feature_len = 1
    
    
class Logon:
#     id, date, user, pc, activity
    col = {name:idx for idx, name in enumerate(["id","date","user","pc","activity"])}
    logon = 1
    logout = 0
    feature_len = 2
    
features_num = 0
for c in [Device, Email, File, Http, Logon]:
    c.feature_shift = features_num
    features_num += c.feature_len + 2
    
ds_dir =  r"D:\simplified_ds"
answers_dir = r"D:\answers"

vectorizer_filename = r"D:\vectorizer.pkl"

user_list_subdir = r"LDAP"
logging.basicConfig(level=logging.DEBUG)

In [ ]:
File.feature_shift
user_computer = {'AAP0352':{6708}}

## 0) Привести набор данных к удобному виду
Пусть каждому пользователю отвечает директория, а файлы data_lig_type в ней отвечают логу пользователя в конкретный день

In [ ]:
out_dir = r"D:\simplified_ds"
in_dir = r"D:\r4.2"
os.makedirs(out_dir, exist_ok=True)
http_path = os.path.join(in_dir, "http.csv")
with open(http_path, "r") as f:
    reader = csv.reader(f, delimiter=',')
    print(next(reader))
    print(next(reader))
    print(next(reader))
    print(next(reader))
    print(next(reader))

In [4]:

# id,date,user,pc
def parse_file(filename, function):
    log_type = os.path.splitext(os.path.basename(filename))[0]
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=',')
        next(reader, None)
        user_records = dict()
        date = ''
        for idx, row in enumerate(reader):
            user = row[2]
            cur_date = datetime.strptime(row[1], '%m/%d/%Y %H:%M:%S').strftime('%Y_%m_%d')
            function(row)
            if cur_date != date:
                for u in user_records:
                    with open(os.path.join(out_dir, u, date +"_"+ log_type), "w") as small_f:
                        small_f.write('\n'.join(user_records[u]))
                date = cur_date
                user_records = dict()
                if date[-1] == '0':
                    logging.info(f"{date} parsed")
                
            user_records.setdefault(user, [])
            user_records[user].append(','.join(row))

In [5]:
def device_f(row):
    row[4] = str(Device.connect if row[4] == "Connect" else Device.disconnect)
    
def email_f(row):
    pass
    
def file_f(row):
    pass
    
def http_f(row):
    pass
    
def logon_f(row):
    row[4] = str(Logon.logon if row[4] == "Logon" else Logon.logout)

In [ ]:
# создадим поддиректории c именами работников, а заодно словари "пользоваель:имеил" и "имеил:пользователь" для всех месяцев
email_user_dict = dict()
user_email_dict = dict()
user_list_dir = os.path.join(in_dir, user_list_subdir)
for filename in os.listdir(user_list_dir):
    cur_email_user = dict()
    cut_user_email = dict()
    
    with open(os.path.join(user_list_dir, filename), "r") as f:
        reader = csv.reader(f, delimiter=',')
        next(reader, None)
        for idx, row in enumerate(reader):
            user_id = row[1]
            user_email = row[2]
            os.makedirs(os.path.join(out_dir, user_id), exist_ok=True)
            cur_email_user[user_email] = user_id
            user_email_dict[user_id] = user_email
            
    email_user_dict[filename] = cur_email_user
    user_email_dict[filename] = cut_user_email
    logging.info(f"{filename} processed")

all_emails = set().union(*[set(email_user_dict[month].keys()) for month in email_user_dict])


select_biggest = lambda x: max(x.items(), key=lambda y: y[1])[0]
# соберем какие компьютеры личные у каких пользователей
#     id,date,user,pc
username_pc_dict = {}
for username in os.listdir(out_dir):
    pc_dict = dict()
    user_dir = os.path.join(out_dir, username)
    for filename in os.listdir(user_dir):
        with open(os.path.join(user_dir, filename), "r") as f:   
            reader = csv.reader(f, delimiter=',')
            pc = row[3]
            pc_dict[pc] = pc_dict.get(pc, 0) + 1
            
    if len(pc_dict) > 1:
        logging.info(f"{pc_dict}")
        
    username_pc_dict[username] = select_biggest(pc_dict)
    
    logging.info(f"{username} processed")

In [ ]:
parse_file(os.path.join(in_dir, "email.csv"), email_f)

In [ ]:
parse_file(os.path.join(in_dir, "device.csv"), device_f)

In [ ]:
parse_file(os.path.join(in_dir, "file.csv"), file_f)

In [ ]:
parse_file(os.path.join(in_dir, "http.csv"), http_f)

In [ ]:
parse_file(os.path.join(in_dir, "logon.csv"), logon_f)

In [ ]:
# загрузим соответствие пользователь - комньютер

## 1) Генерация предложений пользователей (контекст)
Определим список признаков пользователя  
Каждая функция будет возвращать список таплов (дата, номер компьютера, контекст, контент)
#### общие:
- свой/не свой компьютер
- за пределами рабочего дня  
(2 признака)

#### device:
- connect
- disconnet  
(2 признака, контент отсутствует)  

#### email:
- объем
- количество приложений
- человек не из компании
- контент  
(3 признака + контент)

### file:
(1 вариант + контент)

### http:
(1 варианта + контент)

### logon:
- логон
- логофф  
(2 признака)

In [6]:
#     id,date,user,pc,
def has_content(filename):
    return "_email" in filename or "_file" in filename or "_http" in filename

def get_data_pc(line):
    t = datetime.strptime(line[1], '%m/%d/%Y %H:%M:%S').time()
    assert(line[3][:3] == "PC-")
    return (t.hour * 60 + t.minute) * 60 + t.second, int(line[3][3:])

def device_events(user, date):
    #     id,date,user,pc,activity
    device_features = []
    filename = os.path.join(ds_dir, user, date.strftime('%Y_%m_%d') + "_device")
    if not os.path.exists(filename):
        return []
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            features = np.zeros(features_num)
            features[int(row[Device.feature_shift + Device.col["activity"]])] = 1
            device_features.append((*get_data_pc(row), features, None))
    return device_features
Device.events = device_events

def email_events(user, date):
    #     id,date,user,pc,to,cc,bcc,from,size,attachments,content
    def check_email(email_list):
        emails = email_list.split(';')
        for x in emails:
            if x not in emails:
                return True
        return False
        
    email_features = []
    filename = os.path.join(ds_dir, user, date.strftime('%Y_%m_%d') + "_email")
    if not os.path.exists(filename):
        return []
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            features = np.zeros(features_num)
            features[Email.feature_shift] = int(row[Email.col["size"]])
            features[Email.feature_shift+1] = int(row[Email.col["attachments"]])
            features[Email.feature_shift+2] = check_email(row[Email.col["to"]])
            features[Email.feature_shift+3] = check_email(row[Email.col["cc"]])
            email_features.append((*get_data_pc(row), features, row[Email.col["content"]]))
    return email_features


def file_events(user, date):
#     id,date,user,pc,filename,content
    email_features = []
    filename = os.path.join(ds_dir, user, date.strftime('%Y_%m_%d') + "_file")
    if not os.path.exists(filename):
        return []
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            features = np.one(features_num)
            email_features.append((*get_data_pc(row), features, row[File.col["content"]]))
    return email_features


def http_events(user, date):
#     id,date,user,pc,url,content
    http_features = []
    filename = os.path.join(ds_dir, user, date.strftime('%Y_%m_%d') + "_http")
    if not os.path.exists(filename):
        return []
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            features = np.ones(features_num)
            http_features.append((*get_data_pc(row), features, row[Http.col["url"]].replace("/", " ").replace("_", " ") + " " + row[Http.col["content"]]))
    return http_features


def logon_events(user, date):
    #     id, date, user, pc, activity
    logon_features = []
    filename = os.path.join(ds_dir, user, date.strftime('%Y_%m_%d') + "_logon")
    if not os.path.exists(filename):
        return []
    with open(filename, "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            features = np.zeros(features_num)
            features[Logon.feature_shift + int(row[Logon.col["activity"]])] = 1
            logon_features.append((*get_data_pc(row), features, None))
    return logon_features


def merge_features(user, *args):
    all_data = list(itertools.chain(*args))
    all_data.sort(key=lambda x: x[0])
    my_computer = np.array([int(t[1] in username_pc_dict[user]) for t in all_data])
    return [t[0]//600 for t in all_data], np.hstack((my_computer[..., None], np.vstack([t[2] for t in all_data]))), [t[3] for t in all_data]

In [ ]:
user = 'AAE0190'
date = datetime(2010, 1, 5)
features = []
for f in (device_events, email_events, file_events, http_events, logon_events):
    features.append(f(user, date))
    print(features[-1][:3])
    print("---")
merged = merge_features(user, *features)
merged[1].shape

## Предобработка контента
Используется FastText

### Обучение модели

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def sent_gen(max_count=-1):
    for d, _, files in tqdm(os.walk(ds_dir)):
        for filename in filter(has_content, files):
            merged_set = ""
            with open(os.path.join(d, filename), "r") as f:
                reader = csv.reader(f, delimiter=',')
                for row in reader:
                    merged_set = merged_set + " " + row[-1]
            yield merged_set
        max_count -= 1
        if max_count == 0:
            break 
            
vectorizer = CountVectorizer(min_df=5, max_df=0.9, max_features=5000)                    
X = vectorizer.fit_transform(sent_gen())

In [ ]:
with open(vectorizer_filename, 'wb') as fout:
    pickle.dump(vectorizer, fout)
with open()
np.save

## Модель 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

NetConfig = namedtuple('NetConfig', ['operation_len', 'content_len', 'lin0_size', 'encoder_dropout_rate', 'pos_dropout_rate', 'nhead', 'nlayers', 'nhid'])
default_config = NetConfig(
    operation_len = merged[1].shape[1],
    content_len = fasttext_vec_len, 
    lin0_size = 32,
    encoder_dropout_rate = 0.4, 
    pos_dropout_rate = 0.1,
    nhead = 2, 
    dlayers = 4, 
    nhid = 32
)


class MyBERTModel(nn.Module):
    def __init__(self, config):#, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self._linear0 = nn.Linear(config.operation_len + config.content_len, config.lin0_size)
        self._pos_encoder = PositionalEncoding(config.lin0_size, config.dropout_rate)
        self._layer_nm = nn.LayerNorm(config.lin0_size)
        encoder_layers = nn.TransformerEncoderLayer(config.lin0_size, config.nhead, config.nhid, dropout_rate)
        self._transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.init_weights()

    def forward(self, operations, content, src_mask, timestamps=None):
        x = torch.cat((operations, operations), 1)
        x = self._linear0(x)
        x = self._pos_encoder(x)
        output = self.transformer_encoder(x, src_mask)
        return output
    
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, config, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self._dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self._dropout(x)
    
    
class ClassifyModel(nn.Module):
    def __init__(self, config):
        super(ClassifyModel, self).__init__()
        self._BERT = BERTmodel(config)
        self._ln = nn.Linear(config.lin0_size, 2)
        self._softmax = nn.Softmax(dim=1)        
    
    def forward(*args):
        x = self._BERT(*args)[0, :]
        x = self._ln(x)
        x = self._softmax(x)
        return x
    
class MaskedPredicter(nn.Model):
    def __init__(self, config):
        super(MaskedPredicter, self).__init__()
        self._BERT = BERTmodel(config)
        self._ln = nn.Linear(config.lin0_size, config.operation_len + config.content_len) 
    
    def forward(*args):
        x = self._BERT(*args)
        x = self._ln(x)
        return x
    
classify_criterion = nn.CrossEntropyLoss()
